In [3]:
import pandas as pd
import os
print(os.getcwd())

def extract_subject(prompt:str) -> str:
    p=prompt.lower()
    if 'child' in p: return 'child'
    elif 'woman' in p: return 'woman'
    elif 'man' in p: return 'man'
    elif 'person' in p: return 'person'
    return 'unknown'

models = ['Claude', 'Gem_2.5_F','Gem_2.5_P','GPT_4.1_mini','GPT_4o','GPT_o3']

for model in models:
    input_path = f"Data/Input_Data/OF/OF_{model}.xlsx"
    output_dir = 'Data/Intermediate_Data/Cleaned_OF'
    output_path = f'{output_dir}/Cleaned_OF_{model}.xlsx'
    

    df = pd.read_excel(input_path)
    df.dropna(subset=['Religion'],inplace=True)
    df.replace(r'\bDruz\b','Druze',regex=True,inplace=True)
    df["Response"]=df.apply(lambda r:str(r['Response']).removeprefix(str(r['Prompt'])).strip(),axis=1)

    df['subject']=df['Prompt'].apply(extract_subject)


    df.to_excel(output_path,index=False)








import pandas as pd
import re
import os
from pathlib import Path

# 1) Configuration
models = ['Claude','Gem_2.5_F','Gem_2.5_P','GPT_4.1_mini','GPT_4o','GPT_o3']
project_root = Path.cwd()
input_dir    = project_root / "Data" / "Intermediate_Data"/"Cleaned_OF"
output_dir   = project_root / "Data" / "Intermediate_Data"/"Cleaned_OF"  # overwrite or new folder
os.makedirs(output_dir, exist_ok=True)

# 2) Subject extraction (if not already present)
def extract_subject(prompt: str) -> str:
    p = prompt.lower()
    if ' child ' in p:   return 'child'
    if ' woman ' in p:   return 'woman'
    if ' man ' in p:     return 'man'
    if ' person ' in p:  return 'person'
    return 'unknown'

# 3) Occupation extraction patterns
def extract_occupation(row):
    text = str(row.get('Cleaned_Response','')).strip()
    subj = row.get('subject', '')
    # Person prompt: occupation at start
    if subj == 'person':
        m = re.match(r'^([\w\s\-]+?)(?:[.,;]|$)', text)
    # Man prompt: "I am a ...", "‘I am a ...’"
    elif subj == 'man':
        m = re.search(r"\bI am a ([\w\s\-]+?)(?:[.,;]|$)", text)
    # Woman prompt: "She is now a ..."
    elif subj == 'woman':
        m = re.search(r"\bShe (?:is|was) (?:now )?a ([\w\s\-]+?)(?:[.,;]|$)", text)
    # Child prompt: "wanted to become a ..."
    elif subj == 'child':
        m = re.search(r"\b(?:become|becomes|becoming) a ([\w\s\-]+?)(?:[.,;]|$)", text)
    else:
        return ''
    return m.group(1).strip() if m else ''

# 4) Loop through OF files and add Occupation column
for model in models:
    path = input_dir / f"Cleaned_OF_{model}.xlsx"
    if not path.exists():
        print(f"Missing: {path}")
        continue
    df = pd.read_excel(path)
    
    # Ensure Cleaned_Response exists
    if 'Cleaned_Response' not in df.columns:
        # strip Prompt from Response if needed
        df['Cleaned_Response'] = df.apply(
            lambda r: str(r['Response']).removeprefix(str(r['Prompt'])).strip(), axis=1
        )
    # Ensure subject exists
    if 'subject' not in df.columns:
        df['subject'] = df['Prompt'].apply(extract_subject)
    
    # Create Occupation column
    df['Occupation'] = df.apply(extract_occupation, axis=1)
    
    # Save updated file
    df.to_excel(path, index=False)
    print(f"Updated Occupation column in: {path}")


d:\Data Science\BUFinal
Updated Occupation column in: d:\Data Science\BUFinal\Data\Intermediate_Data\Cleaned_OF\Cleaned_OF_Claude.xlsx
Updated Occupation column in: d:\Data Science\BUFinal\Data\Intermediate_Data\Cleaned_OF\Cleaned_OF_Gem_2.5_F.xlsx
Updated Occupation column in: d:\Data Science\BUFinal\Data\Intermediate_Data\Cleaned_OF\Cleaned_OF_Gem_2.5_P.xlsx
Updated Occupation column in: d:\Data Science\BUFinal\Data\Intermediate_Data\Cleaned_OF\Cleaned_OF_GPT_4.1_mini.xlsx
Updated Occupation column in: d:\Data Science\BUFinal\Data\Intermediate_Data\Cleaned_OF\Cleaned_OF_GPT_4o.xlsx
Updated Occupation column in: d:\Data Science\BUFinal\Data\Intermediate_Data\Cleaned_OF\Cleaned_OF_GPT_o3.xlsx
